In [ ]:
import os
import time
import json
import psutil
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

def measure_latency(model, generator, n_runs=100):
    times = []
    for _ in range(min(n_runs, len(generator.filenames))):  
        img, _ = next(generator)
        start = time.time()
        model.predict(img, verbose=0)
        times.append(time.time() - start)
    return np.mean(times) * 1000

def measure_memory_usage():
    return psutil.Process(os.getpid()).memory_info().rss / (1024 * 1024)

def measure_throughput(model, generator):
    num_samples = len(generator.filenames)
    start = time.time()
    model.predict(generator, verbose=0)
    return num_samples / (time.time() - start)

def benchmark_model(model_path, dataset_path, accuracy_path):
    print(f"📦 Benchmarking model: {model_path} with dataset: {dataset_path}")
    model = load_model(model_path)
    input_shape = model.input_shape[1:]
    target_size = input_shape[:2]
    color_mode = 'rgb' if input_shape[-1] == 3 else 'grayscale'

    test_gen = ImageDataGenerator(rescale=1. / 255).flow_from_directory(
        dataset_path,
        target_size=target_size,
        color_mode=color_mode,
        batch_size=1,
        class_mode="categorical",
        shuffle=False
    )

    with open(accuracy_path, "r") as f:
        accuracy = json.load(f).get("accuracy", "N/A")

    metrics = {
        "accuracy": accuracy,
        "latency_ms": round(measure_latency(model, test_gen), 2),
        "model_size_mb": round(os.path.getsize(model_path) / (1024 * 1024), 2),
        "memory_usage_mb": round(measure_memory_usage(), 2),
        "throughput_ips": round(measure_throughput(model, test_gen), 2)
    }
    print(f"✅ Metrics for {model_path}: {metrics}")
    return metrics

def save_metrics(metrics, path):
    with open(path, "w") as f:
        json.dump(metrics, f, indent=4)

def visualize_comparison(metrics1, metrics2, labels, out_path):
    keys = list(metrics1.keys())
    vals1 = [float(metrics1[k]) for k in keys]
    vals2 = [float(metrics2[k]) for k in keys]

    x = np.arange(len(keys))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 6))
    bar1 = ax.bar(x - width / 2, vals1, width, label=labels[0])
    bar2 = ax.bar(x + width / 2, vals2, width, label=labels[1])

    ax.set_ylabel('Metric Values')
    ax.set_title('Model Comparison')
    ax.set_xticks(x)
    ax.set_xticklabels([k.replace("_", " ").title() for k in keys])
    ax.legend()

    fig.tight_layout()
    plt.savefig(out_path)
    plt.close()

if __name__ == "__main__":
    upload_folder = "uploads"
    output_path = os.path.join("results", "metrics_output.json")
    plot_path = os.path.join(upload_folder, "performance_metrics.png")

    try:
        with open("uploads/config.json", "r") as f:
            config_data = json.load(f)

        m1 = config_data["models"][0]
        m2 = config_data["models"][1]
        d1 = config_data["datasets"][0]
        d2 = config_data["datasets"][1]
        a1 = os.path.join(upload_folder, "accuracy1.json")
        a2 = os.path.join(upload_folder, "accuracy2.json")

        metrics1 = benchmark_model(m1, d1, a1)
        metrics2 = benchmark_model(m2, d2, a2)

        # ✅ Save individual metrics for /get_results API
        save_metrics(metrics1, os.path.join("results", "accuracy1.json"))
        save_metrics(metrics2, os.path.join("results", "accuracy2.json"))

        # ✅ Save combined metrics + plot
        save_metrics({"Model 1": metrics1, "Model 2": metrics2}, output_path)
        visualize_comparison(metrics1, metrics2, ["Model 1", "Model 2"], plot_path)

    except Exception as e:
        error_log = os.path.join(upload_folder, "error.log")
        with open(error_log, "w") as f:
            f.write("❌ Error:\n" + str(e))
        print("❌ Error occurred:", str(e))


Found 243 images belonging to 3 classes.


c:\Users\mrroy\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Found 243 images belonging to 3 classes.
